In [28]:
import numpy as np
from collections import Counter
import pandas as pd
#import matplotlin.pyplot as pyplt

In [29]:
# Import and reorder CSV file
# Row 0 was skipped because row 0 was the row label

df = pd.read_csv('HappinessData-1.csv', skiprows=[0], names=['Unhappy/Happy', 'City Services Availability', 'Housing Cost', 
                                                            'Quality of schools', 'Community trust in local police', 
                                                            'Community Maintenance', 'Availability of Community Room '])
df


,Unhappy/Happy,City Services Availability,Housing Cost,Quality of schools,Community trust in local police,Community Maintenance,Availability of Community Room
0,1,5,3,3.0,3.0,5,3
1,0,5,3,3.0,3.0,5,3
2,0,4,3,3.0,3.0,4,4
3,0,4,1,3.0,4.0,4,5
4,1,3,2,4.0,4.0,4,5
...,...,...,...,...,...,...,...
135,1,4,3,3.0,3.0,3,4
136,0,3,4,3.0,3.0,2,3
137,1,3,3,3.0,5.0,5,5
138,1,3,3,1.0,3.0,3,4


In [30]:
# Unhappy/Happy is the class label: as per specifications, move the class label to the last column
# First list the columns and display
labels = list(df.columns)
labels

['Unhappy/Happy',
 'City Services Availability',
 'Housing Cost',
 'Quality of schools',
 'Community trust in local police',
 'Community Maintenance',
 'Availability of Community Room ']

In [31]:
# Save reference to column
class_label = df['Unhappy/Happy']
class_label

0      1
1      0
2      0
3      0
4      1
      ..
135    1
136    0
137    1
138    1
139    1
Name: Unhappy/Happy, Length: 140, dtype: int64

In [32]:
# drop class label column
df.drop('Unhappy/Happy', axis=1, inplace=True)
df

,City Services Availability,Housing Cost,Quality of schools,Community trust in local police,Community Maintenance,Availability of Community Room
0,5,3,3.0,3.0,5,3
1,5,3,3.0,3.0,5,3
2,4,3,3.0,3.0,4,4
3,4,1,3.0,4.0,4,5
4,3,2,4.0,4.0,4,5
...,...,...,...,...,...,...
135,4,3,3.0,3.0,3,4
136,3,4,3.0,3.0,2,3
137,3,3,3.0,5.0,5,5
138,3,3,1.0,3.0,3,4


In [38]:
# add new column to end of table
df['Unhappy/Happy'] = class_label
df

,City Services Availability,Housing Cost,Quality of schools,Community trust in local police,Community Maintenance,Availability of Community Room,Unhappy/Happy
0,5,3,3.0,3.0,5,3,1
1,5,3,3.0,3.0,5,3,0
2,4,3,3.0,3.0,4,4,0
3,4,1,3.0,4.0,4,5,0
4,3,2,4.0,4.0,4,5,1
...,...,...,...,...,...,...,...
135,4,3,3.0,3.0,3,4,1
136,3,4,3.0,3.0,2,3,0
137,3,3,3.0,5.0,5,5,1
138,3,3,1.0,3.0,3,4,1


In [5]:
def euclidian_distance(x1, x2):
    distance = np.sqrt(np.sum((x1-x2)**2))
    return distance

In [6]:
class KNN:
    def __init__(self, k):
        self.k = k
        
    def fit(self, X, y):
        self.x = X
        self.y = y
        
    def predict(self, X):
        predictions = [self._predict(x) for x in X]
        return predictions
    
    def _predict(self, x):
        # Compute Distance
        distances = [euclidian_distance(x, x_train) for x_train in self.X_train]
        
        # Closest K
        np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        
        # majority vote
        most_common = Counter(k_nearest_labels).most_common()
        return most_common